### Studying tensors found by AlphaTensor

In this notebook we download the new optimal tensor decompositions, we calculate the vectors $u_i$, $v_i$ and $w_i$ for each $i$ running over the best found rank of the decomposition.

Due to computational resources, we focus on the simple cases. We randomly suffle the tensors so that we obtain favorable "energy" landscape for the optimization problem. Based on the experiments in this notebook, these orders were relatively easy to find although the search becomes slower the longer the tensors become.

After this we store the tensors in ordered_tensor_factorizations_r file. The letter r means that we are considering tensors in real-valued field instead of the two-element field.

In [19]:
import numpy as np
np.random.seed(0)

# Download factorizations_r.npz from https://github.com/google-deepmind/alphatensor
filename = "files//factorizations_r.npz"
with open(filename, 'rb') as f:
  factorizations = dict(np.load(f, allow_pickle=True))

In [20]:
print(factorizations["2,2,2"])

[[[ 0  1  1  0  1  1  0]
  [ 0  0 -1  1  0  0  0]
  [ 1  1  1  0  1  0  0]
  [-1 -1 -1  0  0  0  1]]

 [[ 0  0  0  0  1  1  0]
  [ 1  1  0  0  1  0  1]
  [ 0  1  1  1  1  0  0]
  [ 0  1  1  0  1  0  1]]

 [[ 0  0  0  1  0  1  0]
  [ 0 -1  0  0  1 -1 -1]
  [-1  1 -1 -1  0  0  0]
  [ 1  0  0  0  0  0  1]]]


In [21]:
# Print available factorizations and their shapes.
data = []
for key in factorizations:
  u, v, w = factorizations[key]
  rank = u.shape[-1]
  assert rank == v.shape[-1] and rank == w.shape[-1]
  print(f'{key}: rank={u.shape[-1]}')
  #print("Maximum absolute valued term in decomposition: ", np.max(np.abs(u)), np.max(np.abs(v)), np.max(np.abs(w)))
  # Calculate number of non-zero elements in each factorization
  #integer_range = len(np.unique(np.concatenate([u, v, w])))
  #print(integer_range)
  #n = int(key.split(",")[0])
  #m = int(key.split(",")[1])
  #k = int(key.split(",")[2])
  #data.append([int(np.ceil(np.log2(integer_range))), n, m, k, int(u.shape[-1])])
#print(data)

2,2,2: rank=7
2,2,3: rank=11
2,2,4: rank=14
2,2,5: rank=18
2,2,6: rank=21
2,2,7: rank=25
2,2,8: rank=28
2,3,3: rank=15
2,3,4: rank=20
2,3,5: rank=25
2,4,4: rank=26
2,4,5: rank=33
2,5,5: rank=40
3,3,3: rank=23
3,3,4: rank=29
3,3,5: rank=36
3,4,4: rank=38
3,4,5: rank=47
3,4,11: rank=103
3,5,5: rank=58
3,5,9: rank=105
3,9,11: rank=225
4,4,4: rank=49
4,4,5: rank=63
4,5,5: rank=76
4,5,9: rank=139
4,5,10: rank=152
4,5,11: rank=169
4,9,10: rank=255
4,9,11: rank=280
4,11,11: rank=343
4,11,12: rank=366
5,5,5: rank=98
5,5,7: rank=134
5,7,9: rank=234
5,7,10: rank=257
5,7,11: rank=280
5,8,9: rank=262
5,8,10: rank=287
5,8,11: rank=317
5,9,9: rank=296
5,9,10: rank=323
5,9,11: rank=358
5,9,12: rank=381
6,7,9: rank=270
6,7,10: rank=296
6,7,11: rank=322
6,8,10: rank=329
6,8,11: rank=365
6,9,9: rank=342
6,9,10: rank=373
6,9,11: rank=411
7,7,9: rank=318
7,7,10: rank=350
7,7,11: rank=384
7,8,9: rank=354
7,8,10: rank=393
7,8,11: rank=432
7,8,12: rank=462
7,9,9: rank=399
7,9,10: rank=441
7,9,11: rank=481
7,

In [4]:
from utils.tensor_utils import get_standard_tensor

for key in factorizations:
    factorization = factorizations[key]
    if factorization[0].shape[-1] > 15:
        continue
    dim_n = int(key.split(",")[0])
    dim_m = int(key.split(",")[1])
    dim_p = int(key.split(",")[2])
    print("Multiplication: ", key)

    # Select the columns from the tensors
    xs = []
    for i in range(factorization[0].shape[-1]):
        xs.append(factorization[0][:, i])
    ys = []
    for i in range(factorization[1].shape[-1]):
        ys.append(factorization[1][:, i])
    zs = []
    for i in range(factorization[2].shape[-1]):
        zs.append(factorization[2][:, i])
    #print(len(xs) == len(ys) == len(zs))

    # This code tests that the standard tensor generation matches 
    # with the tensors used in AlphaTensor paper
    initial_tensor = get_standard_tensor(dim_n, dim_m, dim_p)
    #print(initial_tensor)
    zero_tensor = np.zeros((dim_n*dim_m, dim_m*dim_p, dim_p*dim_n))
    for i in range(len(xs)):
        #print("Non-zero elements: ", np.count_nonzero(zero_tensor))
        #print(zero_tensor)
        if "f2" in filename:
            zero_tensor = np.mod(zero_tensor + np.tensordot(xs[i], np.tensordot(ys[i], zs[i], 0), 0), 2)
        else:
            zero_tensor += np.tensordot(xs[i], np.tensordot(ys[i], zs[i], 0), 0)

    if not np.all(zero_tensor == initial_tensor):
        print("Error in tensor generation")
        print("Initial tensor: ", initial_tensor)

Multiplication:  2,2,2
Multiplication:  2,2,3
Multiplication:  2,2,4
Multiplication:  2,3,3


In [5]:
import pickle

result = {}
keys = ["2,2,3", "2,2,4", "2,3,3", "2,2,5", "2,2,6", "2,2,7", "2,2,8", "2,3,3", "2,3,4", "2,3,5", "2,4,4", "2,4,5", "2,5,5", "3,3,3", "3,3,4", "3,3,5", "3,4,4", "3,4,5"]
f2_keys = ["2,2,3", "2,2,4", "2,2,5", "2,3,3", "2,3,4", "2,3,5", "2,4,4", "2,4,5", "2,5,5", "3,3,3", "3,3,4", "3,3,5", "3,4,4", "3,4,5"]
#keys = ["3,4,11", "3,9,11", "3,5,9", "3,5,5"]
for key in f2_keys:
    factorization = factorizations[key]
    #if factorization[0].shape[-1] > 15 or factorization[0].shape[-1] < 10:
    #    continue
    dim_n = int(key.split(",")[0])
    dim_m = int(key.split(",")[1])
    dim_p = int(key.split(",")[2])
    print("Multiplication: ", key)

    # Select the columns from the tensors
    xs = []
    for i in range(factorization[0].shape[-1]):
        xs.append(factorization[0][:, i])
    ys = []
    for i in range(factorization[1].shape[-1]):
        ys.append(factorization[1][:, i])
    zs = []
    for i in range(factorization[2].shape[-1]):
        zs.append(factorization[2][:, i])

    tensors = [np.tensordot(xs[i], np.tensordot(ys[i], zs[i], 0), 0) for i in range(len(xs))]
    suffled_tensors = None
    i = 0
    while True:
        temp_result = {}
        suffled_tensors = np.random.permutation(tensors)
        initial_tensor = get_standard_tensor(dim_n, dim_m, dim_p)
        increasing_order = []
        decreasing_order = []
        flip = False
        if i % 1000 == 0:
            print("-------------------")
            print("Permutation: ", i)
        i += 1
        for t in suffled_tensors:
            nonzeros = np.count_nonzero(initial_tensor)
            #increasing_order.append(nonzeros)
            #print("Non-zero elements: ", nonzeros)
            if len(increasing_order) > 0:
                # If all previous items in order are smaller than the current one
                #print(all(nonzeros > x for x in increasing_order), not flip)
                if not flip and all(nonzeros > x for x in increasing_order):
                    increasing_order.append(nonzeros)
                elif len(decreasing_order) == 0:
                    flip = True
                    decreasing_order.append(nonzeros)
                    continue
                #print(flip, len(decreasing_order) > 0, all(nonzeros < x for x in decreasing_order))
                if flip and len(decreasing_order) > 0 and all(nonzeros < x for x in decreasing_order):
                    decreasing_order.append(nonzeros)
                elif flip and any(nonzeros > x for x in decreasing_order):
                    #print(nonzeros)
                    #print(increasing_order)
                    #print(decreasing_order)
                    break
            else:
                increasing_order.append(nonzeros)
            
            if "f2" in filename:
                initial_tensor = np.mod(initial_tensor - t, 2)
            else:
                initial_tensor -= t
            
        if len(increasing_order) + len(decreasing_order) == len(suffled_tensors) - 1:
            print("Found a valid order")
            print("Increasing order: ", increasing_order)
            print("Decreasing order: ", decreasing_order)
            break
        
    result[key] = { "tensors": suffled_tensors,
                                "increasing_order": increasing_order,
                                "decreasing_order": decreasing_order,
                                "rank": factorization[0].shape[-1] }

Multiplication:  2,2,3
-------------------
Permutation:  0
Found a valid order
Increasing order:  [12, 18, 20, 24]
Decreasing order:  [24, 20, 13, 12, 6, 4]
Multiplication:  2,2,4
-------------------
Permutation:  0
Found a valid order
Increasing order:  [16, 22, 36, 42, 48, 54, 62, 64, 70]
Decreasing order:  [64, 56, 48, 47]
Multiplication:  2,2,5
-------------------
Permutation:  0
Found a valid order
Increasing order:  [20, 22, 24, 30, 32, 34, 35, 37, 43]
Decreasing order:  [42, 40, 36, 35, 27, 23, 22, 14]
Multiplication:  2,3,3
-------------------
Permutation:  0
Found a valid order
Increasing order:  [18, 28, 32, 34, 44, 46, 56, 74]
Decreasing order:  [68, 32, 20, 18, 8, 6]
Multiplication:  2,3,4
-------------------
Permutation:  0
Found a valid order
Increasing order:  [24, 30, 46, 48, 66, 68, 80, 104, 106]
Decreasing order:  [106, 104, 90, 62, 60, 38, 20, 14, 8, 6]
Multiplication:  2,3,5
-------------------
Permutation:  0
-------------------
Permutation:  1000
-----------------

In [ ]:
with open("files//ordered_tensor_factorizations_f2.pkl", "wb") as f:
    pickle.dump(result, f)